In [82]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D,Dense,Flatten, Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling1D, GlobalAveragePooling1D, LSTM
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model

## Preprocessing

In [41]:
Features=pd.read_csv('features.csv').drop(columns='Unnamed: 0',axis=1)

In [42]:
X = Features.drop(columns='Y',axis=1)
Y = Features['Y']

In [43]:
Y

0        NoStutteredWords
1        NoStutteredWords
2        NoStutteredWords
3        NoStutteredWords
4        NoStutteredWords
               ...       
19670    NoStutteredWords
19671            SoundRep
19672    NoStutteredWords
19673        Interjection
19674    NoStutteredWords
Name: Y, Length: 19675, dtype: object

In [44]:
encoder = OneHotEncoder()

In [45]:
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [50]:
event_mappings=encoder.categories_
event_mappings

[array(['Block', 'Interjection', 'NoStutteredWords', 'Prolongation',
        'SoundRep', 'WordRep'], dtype=object)]

In [55]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)

In [57]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((14756, 20), (14756, 6), (4919, 20), (4919, 6))

In [58]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((14756, 20, 1), (14756, 6), (4919, 20, 1), (4919, 6))

# Arch - 1

In [92]:
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1))) 
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
# model.add(MaxPooling1D(pool_size=(8)))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
opt = keras.optimizers.Adam(learning_rate=0.0001)
# opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_51 (Conv1D)          (None, 20, 256)           2304      
                                                                 
 activation_36 (Activation)  (None, 20, 256)           0         
                                                                 
 conv1d_52 (Conv1D)          (None, 20, 256)           524544    
                                                                 
 batch_normalization_10 (Bat  (None, 20, 256)          1024      
 chNormalization)                                                
                                                                 
 activation_37 (Activation)  (None, 20, 256)           0         
                                                                 
 dropout_13 (Dropout)        (None, 20, 256)           0         
                                                     

In [94]:
model = Sequential()

# Input layer
model.add(layers.Input(shape=(20, 1)))  # Input shape matches the number of MFCC features

# Convolutional layers
model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'))
# model.add(layers.MaxPooling1D(pool_size=2)
model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'))
model.add(layers.MaxPooling1D(pool_size=2))

# Flatten layer
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout for regularization
model.add(layers.Dense(6, activation='softmax'))  # Output layer with 6 categories

In [95]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [96]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

history=model.fit(x_train, y_train, batch_size=16, epochs=200, validation_data=(x_test, y_test), callbacks=[es, rlrp])

Epoch 1/200
923/923 [==============================] - 6s 5ms/step - loss: 1.5368 - accuracy: 0.4910 - val_loss: 1.5001 - val_accuracy: 0.4869 - lr: 1.0000e-04
Epoch 2/200
923/923 [==============================] - 4s 5ms/step - loss: 1.4927 - accuracy: 0.4925 - val_loss: 1.4725 - val_accuracy: 0.4869 - lr: 1.0000e-04
Epoch 3/200
923/923 [==============================] - 4s 5ms/step - loss: 1.4720 - accuracy: 0.4926 - val_loss: 1.4714 - val_accuracy: 0.4881 - lr: 1.0000e-04
Epoch 4/200
923/923 [==============================] - 5s 5ms/step - loss: 1.4574 - accuracy: 0.4927 - val_loss: 1.4539 - val_accuracy: 0.4879 - lr: 1.0000e-04
Epoch 5/200
903/923 [============================>.] - ETA: 0s - loss: 1.4468 - accuracy: 0.4932

KeyboardInterrupt: 